In [1]:
import numpy as np
from IPython.display import display, Math

In [2]:
# Original matrix
A = np.array([[1, 5],
              [0, 10],
              [2, 3]])

# SVD (thin SVD by default)
U, S, Vt = np.linalg.svd(A)

# Build S matrix
Sigma = np.zeros((U.shape[1], Vt.shape[0]))
Sigma[:len(S), :len(S)] = np.diag(S)

# Reconstruct original matrix
A_new = U @ Sigma @ Vt

# LaTeX helper for matrices
def latex_matrix(M):
    return (
        r"\begin{bmatrix}"
        + r" \\ ".join(
            [" & ".join(f"{x:.3f}" for x in row) for row in M]
        )
        + r"\end{bmatrix}"
    )

# Display everything nicely in LaTeX
display(Math(r"\mathbf{A} = " + latex_matrix(A)))
display(Math(r"\mathbf{U} = " + latex_matrix(U)))
display(Math(r"\boldsymbol{\Sigma} = " + latex_matrix(Sigma)))
display(Math(r"\mathbf{V}^\top = " + latex_matrix(Vt)))
display(Math(r"\mathbf{A}_{\mathrm{new}} = " + latex_matrix(A_new)))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>